<a href="https://colab.research.google.com/github/AI4GoodE1/AI4GoodE1/blob/main/DifferentModelsEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

#imports
import numpy as np
import pandas as pd
# import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

#transformers import
from transformers import DistilBertTokenizerFast, DistilBertModel
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from sklearn.metrics import confusion_matrix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.8 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 65.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_trained = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/trained_distilbert_model")
model_20_80 = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/model_report/20_80")
model_50_50 = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/model_report/50_50")
model_untrained = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at /content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/trained_distilbert_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/trained_distilbert_model and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a d

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use i

In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [5]:
# Get data
df = pd.read_csv('https://raw.githubusercontent.com/AI4GoodE1/AI4GoodE1/main/fraud_email_preprocessed.csv')
X = list(df['Text'])
y = list(df['Class'])



In [6]:
final_table = []
untrained_table = []

Model_20_80

In [7]:
# Trained
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.2,random_state=6,shuffle=True)
BATCH_SIZE = 75
EPOCHS = len(X_Test) // BATCH_SIZE
table = []
n,m = 0,BATCH_SIZE
trained_pred_array = [] 
for j in tqdm(range(EPOCHS)):
  batch_info = []
  batch_num = j+1
  batch_info.append(batch_num) # batch number = col 1
  for x in range(n,m):
    test_tokens = tokenizer(X_Test[x], truncation=True, padding=True, return_tensors="tf")
    output = model_20_80(test_tokens)
    prediction = np.argmax(output.logits)
    trained_pred_array.append(prediction)
  true_pos = 0
  false_pos = 0
  true_neg = 0
  false_neg = 0
  for i in range(n,m):
    if y_Test[i] == trained_pred_array[i]:
      if trained_pred_array[i] == 0:
        true_neg += 1
      else:
        true_pos += 1
    else:
      if trained_pred_array[i] == 0:
        false_neg += 1
      else:
        false_pos += 1
  sum_check = true_pos + false_pos + true_neg + false_neg
  accuracy_check = ((true_pos + true_neg) / sum_check) * 100
  accuracy_check = ((true_pos + true_neg) / sum_check) * 100
  recall = (true_pos / (true_pos + false_neg)) * 100
  f1_score = (true_pos / (true_pos + 1/2 * ( false_neg + false_pos))) * 100
  batch_info.append(accuracy_check) # col 6
  batch_info.append(recall) # col 7
  batch_info.append(f1_score) # col 8
  n,m = n + BATCH_SIZE, m + BATCH_SIZE
  table.append(batch_info)
  

  0%|          | 0/27 [00:00<?, ?it/s]

In [8]:
# Untrained
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.2,random_state=6,shuffle=True)
BATCH_SIZE = 75
EPOCHS = len(X_Test) // BATCH_SIZE
utable = []
n,m = 0,BATCH_SIZE
untrained_pred_array = [] 
for j in tqdm(range(EPOCHS)):
  batch_info = []
  batch_num = j+1
  batch_info.append(batch_num)
  for x in range(n,m):
    test_tokens = tokenizer(X_Test[x], truncation=True, padding=True, return_tensors="tf")
    output = model_untrained(test_tokens)
    prediction = np.argmax(output.logits)
    untrained_pred_array.append(prediction)
  true_pos = 0
  false_pos = 0
  true_neg = 0
  false_neg = 0
  for i in range(n,m):
    if y_Test[i] == untrained_pred_array[i]:
      if untrained_pred_array[i] == 0:
        true_neg += 1
      else:
        true_pos += 1
    else:
      if untrained_pred_array[i] == 0:
        false_neg += 1
      else:
        false_pos += 1
  sum_check = true_pos + false_pos + true_neg + false_neg
  accuracy_check = ((true_pos + true_neg) / sum_check) * 100
  recall = (true_pos / (true_pos + false_neg)) * 100
  f1_score = (true_pos / (true_pos + 1/2 * ( false_neg + false_pos))) * 100
  batch_info.append(accuracy_check) # col 6
  batch_info.append(recall) # col 7
  batch_info.append(f1_score) # col 8
  n,m = n + BATCH_SIZE, m + BATCH_SIZE
  utable.append(batch_info)

  0%|          | 0/27 [00:00<?, ?it/s]

In [10]:
#Untrained
df_utable = pd.DataFrame(utable, columns =['BatchNum','Accuracy','Recall','F1_score']) 
df_utable = df_utable.drop(['BatchNum'],axis=1)
uresults = []
df_recallu = df_utable['Recall']
mean_recallu = np.mean(df_recallu)
df_accuracyu = df_utable['Accuracy']
mean_accuracyu = np.mean(df_accuracyu)
df_f1scoreu = df_utable['F1_score']
mean_f1scoreu = np.mean(df_f1scoreu)
uresults.append(mean_recallu)
uresults.append(mean_f1scoreu)
uresults.append(mean_accuracyu)
untrained_table.append(uresults)
untrained_table

[[58.13150134472332, 60.997624879958714, 68.74074074074073]]

In [11]:
#Trained
df_table = pd.DataFrame(table, columns =['BatchNum','Accuracy','Recall','F1_score']) 
df_table = df_table.drop(['BatchNum'],axis=1)
results = []
df_recall = df_table['Recall']
mean_recall = np.mean(df_recall)
df_accuracy = df_table['Accuracy']
mean_accuracy = np.mean(df_accuracy)
df_f1score = df_table['F1_score']
mean_f1score = np.mean(df_f1score)
results.append(mean_recall)
results.append(mean_f1score)
results.append(mean_accuracy)
final_table.append(results)
final_table

[[99.20111962595622, 99.4897398808576, 99.55555555555556]]

Model_70_30


In [12]:
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.3,random_state=0,shuffle=True)
BATCH_SIZE = 75
EPOCHS = len(X_Test) // BATCH_SIZE
table1 = []
n,m = 0,BATCH_SIZE
trained_pred_array = [] 
for j in tqdm(range(EPOCHS)):
  batch_info = []
  batch_num = j+1
  batch_info.append(batch_num) # batch number = col 1
  for x in range(n,m):
    test_tokens = tokenizer(X_Test[x], truncation=True, padding=True, return_tensors="tf")
    output = model_trained(test_tokens)
    prediction = np.argmax(output.logits)
    trained_pred_array.append(prediction)
  true_pos = 0
  false_pos = 0
  true_neg = 0
  false_neg = 0
  for i in range(n,m):
    if y_Test[i] == trained_pred_array[i]:
      if trained_pred_array[i] == 0:
        true_neg += 1
      else:
        true_pos += 1
    else:
      if trained_pred_array[i] == 0:
        false_neg += 1
      else:
        false_pos += 1
  sum_check = true_pos + false_pos + true_neg + false_neg
  accuracy_check = ((true_pos + true_neg) / sum_check) * 100
  recall = (true_pos / (true_pos + false_neg)) * 100
  f1_score = (true_pos / (true_pos + 1/2 * ( false_neg + false_pos))) * 100
  batch_info.append(accuracy_check) # col 6
  batch_info.append(recall) # col 7
  batch_info.append(f1_score) # col 8
  n,m = n + BATCH_SIZE, m + BATCH_SIZE
  table1.append(batch_info)
  

  0%|          | 0/41 [00:00<?, ?it/s]

In [13]:
#Untrained
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.3,random_state=0,shuffle=True)
BATCH_SIZE = 75
EPOCHS = len(X_Test) // BATCH_SIZE
utable = []
n,m = 0,BATCH_SIZE
trained_pred_array = [] 
for j in tqdm(range(EPOCHS)):
  batch_info = []
  batch_num = j+1
  batch_info.append(batch_num)
  for x in range(n,m):
    test_tokens = tokenizer(X_Test[x], truncation=True, padding=True, return_tensors="tf")
    output = model_untrained(test_tokens)
    prediction = np.argmax(output.logits)
    trained_pred_array.append(prediction)
  true_pos = 0
  false_pos = 0
  true_neg = 0
  false_neg = 0
  for i in range(n,m):
    if y_Test[i] == trained_pred_array[i]:
      if trained_pred_array[i] == 0:
        true_neg += 1
      else:
        true_pos += 1
    else:
      if trained_pred_array[i] == 0:
        false_neg += 1
      else:
        false_pos += 1
  sum_check = true_pos + false_pos + true_neg + false_neg
  accuracy_check = ((true_pos + true_neg) / sum_check) * 100
  recall = (true_pos / (true_pos + false_neg)) * 100
  f1_score = (true_pos / (true_pos + 1/2 * ( false_neg + false_pos))) * 100
  batch_info.append(accuracy_check) # col 6
  batch_info.append(recall) # col 7
  batch_info.append(f1_score) # col 8
  n,m = n + BATCH_SIZE, m + BATCH_SIZE
  utable.append(batch_info)

  0%|          | 0/41 [00:00<?, ?it/s]

In [14]:
#Untrained
df_utable = pd.DataFrame(utable, columns =['BatchNum','Accuracy','Recall','F1_score']) 
df_utable = df_utable.drop(['BatchNum'],axis=1)
uresults = []
df_recallu = df_utable['Recall']
mean_recallu = np.mean(df_recallu)
df_accuracyu = df_utable['Accuracy']
mean_accuracyu = np.mean(df_accuracyu)
df_f1scoreu = df_utable['F1_score']
mean_f1scoreu = np.mean(df_f1scoreu)
uresults.append(mean_recallu)
uresults.append(mean_f1scoreu)
uresults.append(mean_accuracyu)
untrained_table.append(uresults)
untrained_table

[[58.13150134472332, 60.997624879958714, 68.74074074074073],
 [57.98970412379029, 60.842429504750434, 68.94308943089429]]

In [15]:
#Trained
df_table1 = pd.DataFrame(table1, columns =['BatchNum','Accuracy','Recall','F1_score']) 
df_table1 = df_table1.drop(['BatchNum'],axis=1)
results1 = []
df_recall1 = df_table1['Recall']
mean_recall1 = np.mean(df_recall1)
df_accuracy1 = df_table1['Accuracy']
mean_accuracy1 = np.mean(df_accuracy1)
df_f1score1 = df_table1['F1_score']
mean_f1score1 = np.mean(df_f1score1)
results1.append(mean_recall1)
results1.append(mean_f1score1)
results1.append(mean_accuracy1)
final_table.append(results1)
final_table

[[99.20111962595622, 99.4897398808576, 99.55555555555556],
 [99.93224932249323, 99.65330231043961, 99.70731707317073]]

Model_50_50

In [16]:
#Trained
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.5,random_state=756,shuffle=True)
BATCH_SIZE = 125
EPOCHS = len(X_Test) // BATCH_SIZE
table2 = []
n,m = 0,BATCH_SIZE
trained_pred_array = [] 
for j in tqdm(range(EPOCHS)):
  batch_info = []
  batch_num = j+1
  batch_info.append(batch_num) # batch number = col 1
  for x in range(n,m):
    test_tokens = tokenizer(X_Test[x], truncation=True, padding=True, return_tensors="tf")
    output = model_trained(test_tokens)
    prediction = np.argmax(output.logits)
    trained_pred_array.append(prediction)
  true_pos = 0
  false_pos = 0
  true_neg = 0
  false_neg = 0
  for i in range(n,m):
    if y_Test[i] == trained_pred_array[i]:
      if trained_pred_array[i] == 0:
        true_neg += 1
      else:
        true_pos += 1
    else:
      if trained_pred_array[i] == 0:
        false_neg += 1
      else:
        false_pos += 1
  sum_check = true_pos + false_pos + true_neg + false_neg
  accuracy_check = ((true_pos + true_neg) / sum_check) * 100
  recall = (true_pos / (true_pos + false_neg)) * 100
  f1_score = (true_pos / (true_pos + 1/2 * ( false_neg + false_pos))) * 100
  batch_info.append(accuracy_check) # col 6
  batch_info.append(recall) # col 7
  batch_info.append(f1_score) # col 9
  n,m = n + BATCH_SIZE, m + BATCH_SIZE
  table2.append(batch_info)
  

  0%|          | 0/41 [00:00<?, ?it/s]

In [17]:
#Untrained
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.5,random_state=756,shuffle=True)
BATCH_SIZE = 125
EPOCHS = len(X_Test) // BATCH_SIZE
utable = []
n,m = 0,BATCH_SIZE
untrained_pred_array = [] 
for j in tqdm(range(EPOCHS)):
  batch_info = []
  batch_num = j+1
  batch_info.append(batch_num) # batch number = col 1
  for x in range(n,m):
    test_tokens = tokenizer(X_Test[x], truncation=True, padding=True, return_tensors="tf")
    output = model_untrained(test_tokens)
    prediction = np.argmax(output.logits)
    untrained_pred_array.append(prediction)
  true_pos = 0
  false_pos = 0
  true_neg = 0
  false_neg = 0
  for i in range(n,m):
    if y_Test[i] == untrained_pred_array[i]:
      if untrained_pred_array[i] == 0:
        true_neg += 1
      else:
        true_pos += 1
    else:
      if untrained_pred_array[i] == 0:
        false_neg += 1
      else:
        false_pos += 1
  sum_check = true_pos + false_pos + true_neg + false_neg
  accuracy_check = ((true_pos + true_neg) / sum_check) * 100
  recall = (true_pos / (true_pos + false_neg)) * 100
  f1_score = (true_pos / (true_pos + 1/2 * ( false_neg + false_pos))) * 100
  batch_info.append(accuracy_check) # col 6
  batch_info.append(recall) # col 7
  batch_info.append(f1_score) # col 8
  n,m = n + BATCH_SIZE, m + BATCH_SIZE
  utable.append(batch_info)

  0%|          | 0/41 [00:00<?, ?it/s]

In [18]:
#Trained
df_table2 = pd.DataFrame(table2, columns =['BatchNum','Accuracy','Recall','F1_score']) 
df_table2 = df_table2.drop(['BatchNum'],axis=1)
results2 = []
df_recall2 = df_table2['Recall']
mean_recall2 = np.mean(df_recall2)
df_accuracy2 = df_table2['Accuracy']
mean_accuracy2 = np.mean(df_accuracy2)
df_f1score2 = df_table2['F1_score']
mean_f1score2 = np.mean(df_f1score2)
results2.append(mean_recall2)
results2.append(mean_f1score2)
results2.append(mean_accuracy2)
final_table.append(results2)
final_table

[[99.20111962595622, 99.4897398808576, 99.55555555555556],
 [99.93224932249323, 99.65330231043961, 99.70731707317073],
 [99.91983809924307, 99.82350567894117, 99.84390243902438]]

In [19]:
#Untrained
df_utable = pd.DataFrame(utable, columns =['BatchNum','Accuracy','Recall','F1_score']) 
df_utable = df_utable.drop(['BatchNum'],axis=1)
uresults = []
df_recallu = df_utable['Recall']
mean_recallu = np.mean(df_recallu)
df_accuracyu = df_utable['Accuracy']
mean_accuracyu = np.mean(df_accuracyu)
df_f1scoreu = df_utable['F1_score']
mean_f1scoreu = np.mean(df_f1scoreu)
uresults.append(mean_recallu)
uresults.append(mean_f1scoreu)
uresults.append(mean_accuracyu)
untrained_table.append(uresults)
untrained_table

[[58.13150134472332, 60.997624879958714, 68.74074074074073],
 [57.98970412379029, 60.842429504750434, 68.94308943089429],
 [59.98496256721494, 63.26590659134976, 70.14634146341464]]

Making the overall table

In [20]:
final_table

[[99.20111962595622, 99.4897398808576, 99.55555555555556],
 [99.93224932249323, 99.65330231043961, 99.70731707317073],
 [99.91983809924307, 99.82350567894117, 99.84390243902438]]

In [21]:
untrained_table

[[58.13150134472332, 60.997624879958714, 68.74074074074073],
 [57.98970412379029, 60.842429504750434, 68.94308943089429],
 [59.98496256721494, 63.26590659134976, 70.14634146341464]]

In [22]:
model_splits = ['20/80','30/70','50/50']
df_final_t = pd.DataFrame(final_table, columns = ['Recall','F1 score','Accuracy']) 
df_untrained_t = pd.DataFrame(untrained_table, columns  = ['Recall','F1 score','Accuracy']) 
df_untrained_t

,Recall,F1 score,Accuracy
0,58.131501,60.997625,68.740741
1,57.989704,60.842430,68.943089
2,59.984963,63.265907,70.146341


In [23]:
df_final_t

,Recall,F1 score,Accuracy
0,99.201120,99.489740,99.555556
1,99.932249,99.653302,99.707317
2,99.919838,99.823506,99.843902


In [26]:
df_newf = df_final_t.rename(index={0: model_splits[0],1:model_splits[1],2:model_splits[2]})
df_newu = df_untrained_t.rename(index={0: model_splits[0],1:model_splits[1],2:model_splits[2]})
df_newf

,Recall,F1 score,Accuracy
20/80,99.201120,99.489740,99.555556
30/70,99.932249,99.653302,99.707317
50/50,99.919838,99.823506,99.843902


In [25]:
# Combine the two tables